In [1]:
!apt-get install openslide-tools
!pip install openslide-python

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libopenslide0
Suggested packages:
  libtiff-tools
The following NEW packages will be installed:
  libopenslide0 openslide-tools
0 upgraded, 2 newly installed, 0 to remove and 34 not upgraded.
Need to get 92.5 kB of archives.
After this operation, 268 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libopenslide0 amd64 3.4.1+dfsg-2 [79.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 openslide-tools amd64 3.4.1+dfsg-2 [12.7 kB]
Fetched 92.5 kB in 1s (144 kB/s)
Selecting previously unselected package libopenslide0.
(Reading database ... 160690 files and directories currently installed.)
Preparing to unpack .../libopenslide0_3.4.1+dfsg-2

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from openslide import open_slide, __library_version__ as openslide_version
import os
from PIL import Image
from skimage.color import rgb2gray

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
path = '/content/drive/MyDrive/ADL/Project/Dataset'

In [5]:
os.listdir(path)

['tumor', 'tumor_mask', 'tumor_xml']

In [6]:
tumor_path = os.path.join(path, 'tumor')
mask_path = os.path.join(path, 'tumor_mask')

In [7]:
tumors = []
for filename in os.listdir(tumor_path):
  tumors.append(os.path.join(tumor_path,filename))
tumors.sort()
masks = []
for filename in os.listdir(mask_path):
  masks.append(os.path.join(mask_path,filename))
masks.sort()

In [8]:
def read_slide(slide, x, y, level, width, height, as_float=False):
    im = slide.read_region((x,y), level, (width, height))
    im = im.convert('RGB') # drop the alpha channel
    if as_float:
        im = np.asarray(im, dtype=np.float32)
    else:
        im = np.asarray(im)
    assert im.shape == (height, width, 3)
    return im

In [25]:
def get_blobs(blob_size = 100, level = 7, gray_threshold):
  blob = 100
  slides = []
  cancerous = []
  for tumor, mask in zip(tumors,masks):
    tumor_img = open_slide(tumor)
    mask_img = open_slide(mask)
    step_width = tumor_img.level_dimensions[7][0] // blob
    step_height = tumor_img.level_dimensions[7][1] // blob
    downsample = tumor_img.level_downsamples[7]

    for i in range(step_width):
      for j in range(step_height):
        x = int(blob * i * downsample)
        y = int(blob * j * downsample)

        tumor_img_blob = read_slide(tumor_img, x, y, level = 7, width = blob, height = blob)
        mask_img_blob = read_slide(mask_img, x, y, level = 7, width = blob, height = blob)

        slides.append(tumor_img_blob)
        if np.sum(mask_img_blob[:,:,0]) > 0:
          cancerous.append(1)
        else:
          cancerous.append(0)
  return slides, cancerous

7 17 (764, 1728) 128.0
119
7 17 (764, 1716) 128.0
119
7 17 (764, 1716) 128.0
119
7 16 (764, 1684) 128.0
112
7 17 (764, 1728) 128.0
119
7 17 (764, 1716) 128.0
119
7 17 (764, 1728) 128.0
119
7 17 (764, 1728) 128.0
119
7 17 (764, 1728) 128.0
119
7 17 (764, 1724) 128.0
119
7 17 (760, 1728) 128.0
119
7 17 (764, 1724) 128.0
119
7 6 (704, 616) 128.0
42
7 8 (736, 868) 128.0
56
7 7 (704, 784) 128.0
49
5 6 (512, 672) 128.0
30
4 4 (480, 420) 128.0
16
9 7 (928, 784) 128.0
63
10 5 (1024, 560) 128.0
50
10 5 (1088, 560) 128.0
50
7 5 (736, 560) 128.0
35


In [18]:
len(cancerous), sum(cancerous)

(1812, 121)